# Week 9 Assingment: What makes a good index?

## Author: Bailey Jørgensen

In [1]:
# %pip install sqlite3

# Connect to the database
import sqlite3
conn = sqlite3.connect("/courses/EDS213/big-fat.sqlite3")
cur = conn.cursor()

# Execute a query
cur.execute("SELECT * FROM Site")
cur.fetchone()

('barr', 'Barrow', 'Alaska, USA', 71.3, -156.6, 220.4)

In [5]:
# Query we will be working with for the assignment
cur.execute("""SELECT Nest_ID
    FROM Bird_nests
    WHERE Site = 'nome' AND
          Species = 'ruff' AND
          Year = 1983 AND
          Observer = 'cbishop' AND
          ageMethod = 'float';""")

result = cur.fetchone()

print("Nest ID: ", result)

Nest ID:  ('gen511190',)


### Part 1:
Answer the following questions.

- Is there already an index on the Bird_nests table? If so, what is that index and will SQLite use it in the above query? Why or why not?

In [13]:
# Check to see if there is are indexes 
cur.execute("""PRAGMA index_list(Bird_nests);""")
result = cur.fetchall()

print(result)

[(0, 'sqlite_autoindex_Bird_nests_1', 1, 'pk', 0)]


In [12]:
# See if this index is used in this query
cur.execute("""EXPLAIN QUERY PLAN 
    SELECT Nest_ID
    FROM Bird_nests
    WHERE Site = 'nome' AND
          Species = 'ruff' AND
          Year = 1983 AND
          Observer = 'cbishop' AND
          ageMethod = 'float';""")
result = cur.fetchall()
print(result)

[(2, 0, 0, 'SCAN Bird_nests')]


It looks like, based on the searches above, that there is one index already on the Bird_nests table. However, that index is not used in the Query for this assingment, because the query plan involves "SCAN Bird_nests", which is the entire table.

- Will adding an index on a column not mentioned in the WHERE clause be used by the database? Why or why not?

Since the existing index does not live on any of the columns mentioned by the WHERE clause, it is not used during the query. This makes sense, since the database has no need to use the indexes and columns not called in the query. 

### Part 2: